<a href="https://colab.research.google.com/github/CE334/CE334_230586_Kulshreshth_Chikara/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gravimetry Network Adjustment with Drift
# Batch Data: CG-6_0251_G5-CE334.csv
# Course Assignment Submission

In [1]:
import numpy as np
import pandas as pd

In [5]:
# 1. Load CSV Data
file_path = "data.csv"   # Upload this file to Colab first
df = pd.read_csv(file_path)

print("Total Observations:", len(df))
print(df.head())


Total Observations: 30
  Station        Date      Time   CorrGrav  Line  StdDev  StdErr    RawGrav  \
0     FF1  2006-06-14  09:58:05  2938.1049     0  0.0122  0.0022  2919.8542   
1     FF1  2006-06-14  09:58:35  2938.1060     0  0.0110  0.0020  2919.8561   
2     FF1  2006-06-14  09:59:05  2938.1043     0  0.0111  0.0020  2919.8545   
3     TF1  2006-06-14  10:04:28  2936.0332     0  0.0114  0.0021  2917.7870   
4     TF1  2006-06-14  10:04:58  2936.0323     0  0.0124  0.0023  2917.7858   

     X    Y  ...  DriftCorr  MeasurDur  InstrHeight    LatUser    LonUser  \
0 -5.3  4.4  ...    18.3922         30            0  26.515766  80.230446   
1 -3.1  4.9  ...    18.3922         30            0  26.515766  80.230446   
2 -4.5  5.1  ...    18.3922         30            0  26.515766  80.230446   
3 -5.2  4.6  ...    18.3922         30            0  26.515877  80.230370   
4 -7.2  5.3  ...    18.3922         30            0  26.515877  80.230370   

   ElevUser     LatGPS     LonGPS  Elev

In [6]:
# TASK 2: Observation Differencing (Equation 2)

# Convert Time to Decimal Hours
def time_to_decimal(t):
    h, m, s = map(int, t.split(":"))
    return h + m/60 + s/3600

# Apply conversion
df["t_decimal"] = df["Time"].apply(time_to_decimal)

# Reference observation (first measurement)
y1 = df["CorrGrav"].iloc[0]
t1 = df["t_decimal"].iloc[0]

# Compute differenced values
df["delta_y"] = df["CorrGrav"] - y1
df["delta_t"] = df["t_decimal"] - t1

print("Differenced Observations Created:")
df[["Station", "delta_y", "delta_t"]].head()


Differenced Observations Created:


,Station,delta_y,delta_t
0,FF1,0.0000,0.000000
1,FF1,0.0011,0.008333
2,FF1,-0.0006,0.016667
3,TF1,-2.0717,0.106389
4,TF1,-2.0726,0.114722


In [7]:
# TASK 3: Weight Matrix Construction

sigma = df["StdDev"].values

# Weight matrix
W = np.diag(1 / sigma**2)

print("Weight Matrix Shape:", W.shape)
print("\nFirst 5 Weights:")
print(np.diag(W)[:5])


Weight Matrix Shape: (30, 30)

First 5 Weights:
[6718.6240258  8264.46280992 8116.22433244 7694.6752847  6503.64203954]


In [13]:
# TASK 4: Correct Design Matrix with Drift

stations = df["Station"].unique()
m = len(stations)
n = len(df)

# Unknowns: (m-1) gravity differences + drift
u = (m - 1) + 1

# Reference station = first one
ref_station = stations[0]

# Map non-reference stations
station_index = {st: i for i, st in enumerate(stations[1:])}

# Build A matrix
A = np.zeros((n, u))

for k in range(n):
    st = df["Station"].iloc[k]
    dt = df["delta_t"].iloc[k]

    # Gravity parameter (skip reference)
    if st != ref_station:
        idx = station_index[st]
        A[k, idx] = 1

    # Drift is last column
    A[k, -1] = dt

print("Correct A shape:", A.shape)

l = df["delta_y"].values.reshape(-1, 1)
sigma = df["StdDev"].values
W = np.diag(1 / sigma**2)

N = A.T @ W @ A
U = A.T @ W @ l

x_hat = np.linalg.inv(N) @ U
v = A @ x_hat - l

r = n - u
sigma0_sq = (v.T @ W @ v) / r

Cov_x = sigma0_sq * np.linalg.inv(N)
std_params = np.sqrt(np.diag(Cov_x))

param_names = list(stations[1:]) + ["Drift d"]

print("\nFINAL RESULTS\n")
for i, name in enumerate(param_names):
    print(f"{name:6s} = {x_hat[i,0]:10.6f} ± {std_params[i]:.6f}")


Correct A shape: (30, 10)

FINAL RESULTS

TF1    =  -2.066227 ± 0.003148
FF2    =  -0.003489 ± 0.005665
TF2    =  -2.068555 ± 0.008582
5F1    =  -4.262093 ± 0.010775
TF3    =  -2.075957 ± 0.012967
5F2    =  -4.266699 ± 0.015154
7F1    =  -6.444107 ± 0.017591
5F3    =  -4.275934 ± 0.019858
7F2    =  -6.449315 ± 0.022089
Drift d =  -0.051320 ± 0.027369


In [14]:
# TASK 4: Variance Computation

# Residuals
v = A @ x_hat - l

# A priori variance
sigma0_apriori = 1

# A posteriori variance
sigma0_sq = (v.T @ W @ v) / r

print("A Priori Variance σ0² =", sigma0_apriori)
print("A Posteriori Variance σ̂0² =", sigma0_sq[0,0])


A Priori Variance σ0² = 1
A Posteriori Variance σ̂0² = 0.005662280700837574


In [15]:
# TASK 5: Variance–Covariance Matrix

Cov_x = sigma0_sq * np.linalg.inv(N)

print("\nVariance–Covariance Matrix:")
print(Cov_x)

# Standard deviation of parameters
std_params = np.sqrt(np.diag(Cov_x))

print("\nStandard Deviations of Estimated Parameters:")
print(std_params)



Variance–Covariance Matrix:
[[ 9.90958091e-06  1.74817929e-05  2.64761316e-05  3.33140992e-05
   4.01057911e-05  4.69041419e-05  5.44624922e-05  6.14941110e-05
   6.83991729e-05 -8.48200847e-05]
 [ 1.74817929e-05  3.20976087e-05  4.81889882e-05  6.06347165e-05
   7.29962189e-05  8.53698409e-05  9.91267317e-05  1.11924923e-04
   1.24492769e-04 -1.54380335e-04]
 [ 2.64761316e-05  4.81889882e-05  7.36553270e-05  9.18311265e-05
   1.10552591e-04  1.29292411e-04  1.50127187e-04  1.69510016e-04
   1.88543988e-04 -2.33808631e-04]
 [ 3.33140992e-05  6.06347165e-05  9.18311265e-05  1.16093839e-04
   1.39104913e-04  1.62684650e-04  1.88900406e-04  2.13289221e-04
   2.37239080e-04 -2.94194184e-04]
 [ 4.01057911e-05  7.29962189e-05  1.10552591e-04  1.39104913e-04
   1.68151962e-04  1.95850908e-04  2.27411229e-04  2.56772152e-04
   2.85604631e-04 -3.54171080e-04]
 [ 4.69041419e-05  8.53698409e-05  1.29292411e-04  1.62684650e-04
   1.95850908e-04  2.29635483e-04  2.65959809e-04  3.00297716e-04
   3

In [16]:
# Correct Design Matrix A

# Observation vector
l = df["delta_y"].values.reshape(-1, 1)

# Weight matrix
sigma = df["StdDev"].values
W = np.diag(1 / sigma**2)

# Normal equation
N = A.T @ W @ A
U = A.T @ W @ l

# Solve for parameters
x_hat = np.linalg.inv(N) @ U

# Residuals
v = A @ x_hat - l

# Redundancy
n_obs = len(df)
u_params = A.shape[1]
r = n_obs - u_params

# Posterior variance
sigma0_sq = (v.T @ W @ v) / r

# Covariance matrix
Cov_x = sigma0_sq * np.linalg.inv(N)

# Standard deviations
std_params = np.sqrt(np.diag(Cov_x))

print("x_hat shape:", x_hat.shape)
print("std_params shape:", std_params.shape)
param_names = list(stations[1:]) + ["Drift d"]

print("\n===============================")
print("FINAL ADJUSTED PARAMETERS")
print("===============================")

for i, name in enumerate(param_names):
    print(f"{name:6s} = {x_hat[i,0]:10.6f} ± {std_params[i]:.6f}")



x_hat shape: (10, 1)
std_params shape: (10,)

FINAL ADJUSTED PARAMETERS
TF1    =  -2.066227 ± 0.003148
FF2    =  -0.003489 ± 0.005665
TF2    =  -2.068555 ± 0.008582
5F1    =  -4.262093 ± 0.010775
TF3    =  -2.075957 ± 0.012967
5F2    =  -4.266699 ± 0.015154
7F1    =  -6.444107 ± 0.017591
5F3    =  -4.275934 ± 0.019858
7F2    =  -6.449315 ± 0.022089
Drift d =  -0.051320 ± 0.027369
